In [1]:
import pandas as pd

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F #take a long time 

d:\mini\envs\pine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_3_review = pd.read_csv('โซน คลองหนึ่ง\df_review_combined.csv')
df_1_review = pd.read_csv('โซน หลักหก\df_review_combined11.csv')
df_2_review = pd.read_csv('โซน รังสิต\df_review_combined11.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\eieiz\AppData\Local\Temp\ipykernel_14464\3619220367.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_3_review = pd.read_csv('โซน คลองหนึ่ง\df_review_combined.csv')
C:\Users\eieiz\AppData\Local\Temp\ipykernel_14464\3619220367.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_1_review = pd.read_csv('โซน หลักหก\df_review_combined11.csv')
C:\Users\eieiz\AppData\Local\Temp\ipykernel_14464\3619220367.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_2_review = pd.read_csv('โซน รังสิต\df_review_combined11.csv')


https://huggingface.co/poom-sci/WangchanBERTa-finetuned-sentiment

In [4]:
import numpy as np

In [5]:
from transformers import pipeline

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "poom-sci/WangchanBERTa-finetuned-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)




Device set to use cuda:0


In [7]:
classifier('dddddddddddddddddddddddd', truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'label': 'neu', 'score': 0.803070068359375}]

In [8]:
df_1_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_review_main  609 non-null    object 
 1   review_rating   609 non-null    float64
 2   review_text     609 non-null    object 
dtypes: float64(1), object(2)
memory usage: 22.2+ KB


In [9]:
df_1_review['review_text'] = df_1_review['review_text'].astype(str)
df_2_review['review_text'] = df_2_review['review_text'].astype(str)
df_3_review['review_text'] = df_3_review['review_text'].astype(str)


In [10]:
for i in df_1_review['review_text']:
    if i == 'nan':
        print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [11]:
df_1_review['review_text'] = df_1_review['review_text'].replace('nagative', 'negative')
df_2_review['review_text'] = df_2_review['review_text'].replace('nagative', 'negative')
df_3_review['review_text'] = df_3_review['review_text'].replace('nagative', 'negative')


In [12]:
df_1_review[df_1_review['review_text'] == 'negative']

,id_review_main,review_rating,review_text
382,review_320,1.0,negative
614,review_345,2.0,negative
867,review_405,1.0,negative
937,review_407,1.0,negative


In [13]:
from tqdm import tqdm

def apply_sentiment(df, pipe):
    sentiments = []
    for text in tqdm(df['review_text'], desc="Processing Sentiment"):
        text = str(text).strip()  # ✅ แปลงเป็น string และลบช่องว่าง

        if text.lower() == 'nan':  # ✅ ถ้าเดิมเป็น NaN
            sentiments.append(np.nan)
        elif text == 'positive':
            sentiments.append('pos')
        elif text == 'negative':
            sentiments.append('neg')
        else:
            try:
                result = pipe(text, truncation=True)[0]['label']
                sentiments.append(result)
            except Exception as e:
                result = pipe(text[:300], truncation=True)[0]['label']
                sentiments.append(result)
    df['sentiment'] = sentiments
    return df


In [14]:
df_1_review = apply_sentiment(df_1_review, classifier)
df_2_review = apply_sentiment(df_2_review, classifier)
df_3_review = apply_sentiment(df_3_review, classifier)

Processing Sentiment:   0%|          | 0/943 [00:00<?, ?it/s]

Processing Sentiment: 100%|██████████| 1165/1165 [00:08<00:00, 145.19it/s]


In [15]:
df_1_review

,id_review_main,review_rating,review_text,sentiment
0,review_300,1.0,พนง.หน้าตาไม่ยิ้มแย้ม ไม่ถามรหัสสมาชิกเลย ภาพร...,neg
1,review_300,1.0,พนักงานทำช้ามากๆภาพรวมแย่มากๆ,neg
2,review_300,5.0,อร่อยมากกกก รสชาติดี ลองเกือบหมดแล้ว💘💘 …,pos
3,review_301,5.0,ยินดีให้บริการ เราจะเสิร์ฟทั้งของหวานและความอร...,pos
4,review_301,5.0,พ่อค้าน้ารักเป็นกันเองทำอาหารอร่อย แถมเก่งด้วย,pos
...,...,...,...,...
938,review_407,5.0,ขอบคุณค่ะ,pos
939,review_407,4.0,ขอบคุฯค่ะ,pos
940,review_407,5.0,ขอบคุณค่ะ,pos
941,review_407,5.0,ขอบคุณค่ะ,pos


In [16]:
df_1_review.to_csv('df_1_review_clean.csv', index=False,encoding='utf-8-sig')
df_2_review.to_csv('df_2_review_clean.csv', index=False,encoding='utf-8-sig')
df_3_review.to_csv('df_3_review_clean.csv', index=False,encoding='utf-8-sig')